In [1]:
# Load model directly
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset


/opt/homebrew/anaconda3/envs/torch-base/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-410m")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.bos_token = tokenizer.eos_token
print(tokenizer.pad_token)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<|endoftext|>


In [4]:
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-410m")
model.to("mps")
model.device

device(type='mps', index=0)

In [5]:
raw_datasets = load_dataset("iamtarun/python_code_instructions_18k_alpaca",split="train")

In [6]:
raw_datasets = raw_datasets.train_test_split(test_size=0.1)

In [7]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 16750
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 1862
    })
})


In [8]:
def preprocess_function(data):
    return tokenizer(data["prompt"])


tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=raw_datasets["test"].column_names,
)

Map (num_proc=4): 100%|██████████| 1862/1862 [00:00<00:00, 7350.39 examples/s]


In [9]:
print(tokenized_datasets)
print(tokenized_datasets["train"][0])
print(tokenizer.decode(tokenized_datasets["train"][2]["input_ids"]))

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 16750
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1862
    })
})
{'input_ids': [30003, 310, 271, 9775, 326, 8631, 247, 4836, 15, 19566, 247, 2380, 326, 20420, 29141, 253, 2748, 15, 187, 187, 4118, 41959, 27, 187, 9395, 247, 15548, 2086, 326, 3936, 247, 6197, 285, 6548, 247, 1618, 273, 3000, 275, 30156, 474, 1340, 15, 187, 187, 4118, 19832, 27, 187, 29710, 566, 27, 50276, 510, 3158, 8516, 30013, 16780, 689, 253, 22658, 4370, 15, 187, 187, 4118, 24882, 27, 187, 36817, 426, 346, 510, 3158, 8516, 30013, 16780, 689, 253, 22658, 4370, 449, 187, 187, 4, 43531, 6197, 715, 3000, 187, 12113, 426, 6197, 15, 9148, 1082, 187, 187, 4, 17399, 1618, 273, 3000, 355, 20376, 17859, 187, 12113, 15, 15227, 1082, 187, 187, 4, 18312, 20045, 1618, 273, 3000, 187, 3845, 9, 12113, 10, 187, 187, 4, 24882, 27, 14412, 33167, 1383, 686, 21428, 40219, 686, 15736,

In [10]:
block_size = 256


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4): 100%|██████████| 1862/1862 [00:00<00:00, 6815.39 examples/s]


In [ ]:
print(lm_datasets)
for i in range(5):
    print(tokenizer.decode(lm_datasets["train"][i]["input_ids"]))
    print("-----------------------------------------------")
# print(tokenizer.decode(lm_datasets["train"][0]["input_ids"]))

In [ ]:
input_ids = tokenizer(["Write a Python Code for calculating 1 + 1"], return_tensors="pt").to('mps')
print("input_ids", input_ids)
generated_ids = model.generate(**input_ids, max_new_tokens=100, repetition_penalty=2.)

In [ ]:
print(tokenizer.batch_decode(generated_ids)[0])

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [11]:
training_args = TrainingArguments(
    output_dir="save/3/",
    report_to='tensorboard',
    seed=4328,
    logging_steps=10,
    per_device_train_batch_size=4,
    learning_rate=5e-7,
    warmup_steps=100,
    num_train_epochs=1,
    save_total_limit= 3,
    save_strategy="steps",
    save_steps=300,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    eval_steps=300,
)


In [12]:
import evaluate
import numpy as np
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
trainer = Trainer(
    model=model,
    train_dataset=lm_datasets['train'],
    eval_dataset=lm_datasets['test'],
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
)

trainer.train(resume_from_checkpoint=True)

  7%|▋         | 210/2852 [00:13<00:26, 98.47it/s]

{'loss': 1.1376, 'grad_norm': 7.879387855529785, 'learning_rate': 4.631837307152875e-06, 'epoch': 0.07}


  8%|▊         | 220/2852 [00:26<10:15,  4.27it/s]

{'loss': 1.2102, 'grad_norm': 6.814539909362793, 'learning_rate': 4.614305750350632e-06, 'epoch': 0.08}


  8%|▊         | 230/2852 [00:38<36:29,  1.20it/s]

...

100%|█████████▉| 2850/2852 [1:42:16<00:02,  1.26s/it]

{'loss': 0.9505, 'grad_norm': 4874597.5, 'learning_rate': 3.506311360448808e-09, 'epoch': 1.0}


100%|██████████| 2852/2852 [1:42:20<00:00,  2.15s/it]

{'train_runtime': 6140.7023, 'train_samples_per_second': 2.322, 'train_steps_per_second': 0.464, 'train_loss': 1.0136234136650975, 'epoch': 1.0}


TrainOutput(global_step=2852, training_loss=1.0136234136650975, metrics={'train_runtime': 6140.7023, 'train_samples_per_second': 2.322, 'train_steps_per_second': 0.464, 'train_loss': 1.0136234136650975, 'epoch': 1.0})

In [14]:
model.save_pretrained("save/completed/2")